In [2]:
import numpy as np

In [103]:
A = [0, 1]

P_wait = np.array([
    [0.97, 0.01, 0.005, 0.005, 0.0025, 0.0025, 0.002, 0.002, 0, 0.001],
    [0.015, 0.965, 0.005, 0.005, 0.0025, 0.0012, 0.0012, 0.0025, 0.0013, 0.0013],
    [0.005, 0.005, 0.96, 0.01, 0.005, 0.005, 0.0025, 0.0025, 0.0025, 0.0025],
    [0, 0.005, 0.005, 0.95, 0.01, 0.01, 0.005, 0.005, 0.005, 0.005],
    [0, 0, 0.005, 0.005, 0.94, 0.02, 0.01, 0.01, 0.004, 0.006],
    [0, 0, 0, 0.005, 0.005, 0.93, 0.02, 0.02, 0.013, 0.007],
    [0, 0, 0, 0, 0.005, 0.007, 0.93, 0.03, 0.02, 0.008],
    [0, 0, 0, 0, 0, 0.005, 0.001, 0.914, 0.04, 0.04],
    [0, 0, 0, 0, 0, 0, 0.01, 0.01, 0.9, 0.08],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])

S = list(range(0,11))  # 10 is transplant, 9 is death

rT = dict(zip(list(range(0,11)), [100,96,90,84,76,68,56,40,25,0, 0]))

rW = dict(zip(list(range(0, 11)), [0.99,0.98,0.97,0.96,0.93,0.9,0.87,0.84,0.8,0, 0]))
lam = 0.9998
epsilon = 1e-6

In [104]:
def P(j, s, a):

    if s == 10 and j == 10:  # Transplant Absorbing State
        p = 1
    elif a == 1 and j == 10:  # Transplant (from non-transplant state)
        p = 1
    elif a == 0 and j != 10 and s != 10:
        p = P_wait[s, j]
    else:
        p = 0

    return p

def r(s, a):
    if a == 0:
        rwd = rW[s]
    elif a == 1:
        rwd = rT[s]
    else:
        rwd = 0

    return rwd

span = lambda v_: max(v_) - min(v_)

In [ ]:
v_n = np.array([max(r(s, a) for a in A) for s in S])
v_n1 = np.empty(len(S))
n=1
optimal = False
while not optimal:

    for s in S:
        v_n1[s] =  max(r(s, a_) + (lam * sum(P(j, s, a_)* v_n[j] for j in S)) for a_ in A)

    print('iteration ', n)

    if span(v_n1 - v_n) >= ((1-lam)/lam) * epsilon:
        optimal = True

Lv = np.array([max(r(s, a_) + (lam * sum(P(j, s, a_)* v_n[j] for j in S)) for a_ in A) for s in S])

In [116]:
d_ep = np.empty(len(S))
v_ep = np.empty(len(S))

for s in S:
    d_ep[s] = np.argmax([r(s, a_) + (lam * sum(P(j, s, a_) * v_n1[j] for j in S)) for a_ in A])
    v_ep[s] = v_n1[s] + ((lam / (1-lam)) * min(Lv - v_n))

In [117]:
print('v = ', v_ep)
print('d = ', d_ep)

v =  [100.3521236   96.44220374  90.1946515   84.          76.
  68.          56.          40.          25.           0.
   0.        ]
d =  [0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0.]
